In [ ]:
import pandas as pd
import os
import numpy as np
import warnings
import keras.backend as K
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras import backend as K
import numpy as np
import tensorflow as tf
warnings.filterwarnings("ignore")


conduct_path="E:/code"
filename = "foam1.csv"


#define a function to load csv data set, by changing the directory name conduct_path and file name filename, the function can read different csv files
def load_conduct_data():
    csv_path = os.path.join(conduct_path, filename)
    return pd.read_csv(csv_path)

def r2(y_test, y_pred):
    a = K.square(y_pred - y_test)
    b = K.sum(a)
    c = K.mean(y_test)
    d = K.square(y_test - c)
    e = K.sum(d)
    f = 1 - b/e
    return f

In [ ]:
#load the file and use conduct to represent the loaded content
conduct = load_conduct_data()

#change the loaded content to numpy format
c = np.array(conduct)

#select the first 131 columns as features, i.e. [0,130]
X = c[:,:131]

#select the label, the label is the output, in this data set, 131, 132, 133 list the thermal conductivity of the foam in three directions, there is no special direction, so any direction of the feature is selected as the processing index
y = c[:,131:132]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#show the correlation coefficient between the features and the label
corr = conduct.corr().abs()
sns.heatmap(data=corr)
plt.title("Coefficients")
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Get the absolute value of the correlation coefficient of the thermal conductivity in the X direction
cccc = conduct.corr().abs()['CX']
value = []

#Put the correlation coefficient of each feature into the value list
for v in cccc:
    value.append(v)
# value.sort()
array = []

#Put the name of each feature into the array list
for column in conduct:
    array.append(column)

In [ ]:
dict_features = {}
cor = []
features_name = []
labels = ['CX','CY','CZ']
#Build a dictionary dict_features. The dictionary uses a key-value pair. The array is used as the key and the value is used as the value. It is added to the dictionary. At the same time, the value list is copied to the cor list.
for i in range(len(array)):
    if not np.isnan(value[i]) and not array[i] in labels:
        dict_features[array[i]] = value[i]
        features_name.append(array[i])
        cor.append(value[i])


In [ ]:
dict_v2n = {}

#Remove features with a correlation coefficient of None from the dictionary, which is equivalent to cleaning the data
for i in range(len(dict_features)):
    co = str(cor[i])
    if(dict_v2n.get(co) == None):
        dict_v2n[co] = features_name[i]
        
#Sort the correlation coefficient from large to small, where sort() is sorted from small to large, reverse() reverses the process, and finally obtains the correlation coefficient sorted from large to small
cor.sort()
cor.reverse()

In [ ]:
features_list = []
#Add the names of the correlation coefficients to the features_list list in the order of the correlation coefficients from large to small
for i in range(len(cor)):
    co = str(cor[i])
    if not dict_v2n[co] in features_list:
        features_list.append(dict_v2n[co])

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
import itertools
import _thread
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))

In [ ]:
final_sublist = []

#forward select is responsible for calculating the error. The function is to add the feature at the specified index to the feature subset sublist, and use svm to train the feature subset, and return the error of the model
def get_err(sublist,index):
    sublist.append(features_list[index])
    X = np.array(conduct[sublist])
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 32, shuffle = True)
    svr_pre = regr.fit(X_train,y_train).predict(X_test)
    return mean_absolute_percentage_error(y_test,svr_pre)

#the entry of forward select
def features_forward_select(sublist,delay_times):
    global min_error
    global final_sublist
    left_index = 0
    while left_index < len(features_list):     
        mape_err = get_err(sublist,left_index)
        #if the error of the current subset is less than the error of the minimum subset, then the current subset is the minimum subset, and the current error is the minimum error (the minimum subset refers to the feature subset with the minimum error)
        if mape_err < min_error:
            min_error = mape_err
            final_sublist = sublist[:]
            print(str(len(sublist)) + "," + str(min_error))
            print(sublist)
        #if the error of the current subset is not less than the error of the minimum subset, then remove the last feature of the current subset, which is equivalent to returning to the state when the last feature was added
        #and start a depth search from this state until the maximum number of the maximum feature subset
        else:
            sublist.remove(sublist[len(sublist) - 1])
            ok,index_return,list_return = delay_seek(sublist,delay_times,left_index)
            if ok:
                left_index = index_return
                sublist = list_return
            else:
                break
        left_index += 1
    return sublist
            
# deep search function, delay_times represents the maximum number of searches, delay_times + init_index represents the maximum threshold for the search
def delay_seek(sublist,delay_times,init_index):
    global min_error
    global final_sublist
    if delay_times <= 0 or delay_times >= len(features_list):
        print("Error, delay_times must be a positive number!")
        return False,None,None
    for i in range(init_index,len(features_list)):
        mape_err = get_err(sublist,i)
        if mape_err < min_error:
            min_error = mape_err
            final_sublist = sublist[:]
            print(str(len(sublist)) + "," + str(min_error))
            print(sublist)
            return True, i, sublist
        else:
            sublist.remove(sublist[len(sublist) - 1])
    delay_times -= 1
    if(delay_times > 0 and init_index < len(features_list)):
        sublist.append(features_list[init_index])
        ok,index_return,list_return = delay_seek(sublist,delay_times,init_index + 1)
        if ok:
            return ok,index_return,list_return
        else:
            sublist.remove(sublist[len(sublist) - 1])
    return False,None,None


# remove_err is responsible for calculating the error. The function is to remove the feature at the specified index from the feature subset sublist, and use svm to train the feature subset, and return the error of the model
def remove_err(sublist,index):
    sublist.remove(sublist[index])
    X = np.array(conduct[sublist])
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 32, shuffle = True)
    svr_pre = regr.fit(X_train,y_train).predict(X_test)
    return r2_score(y_test,svr_pre),mean_absolute_percentage_error(y_test,svr_pre)

# the entry of backward select
def features_backward_select(sublist,delay_times):
    global max_r2
    global min_error
    global final_sublist
    right_index = len(sublist)
    while right_index >= 0:
        right_index -= 1
        feature = sublist[right_index]
        r2,mape_err = remove_err(sublist,right_index)
        print("nowlist="+str(len(sublist)))

        if mape_err < min_error and r2 > max_r2 :
            min_error = mape_err
            max_r2 = r2
            final_sublist = sublist[:]
            print(str(len(sublist)) + ",min_error=" + str(min_error) + ",max_r2=" + str(max_r2))
        else:
            sublist.insert(right_index,feature)
            ok,index_return,list_return = delay_seek_backword(sublist,delay_times,right_index)
            if ok:
                right_index = index_return
                sublist = list_return
            else:
                break
    return sublist

#deep search in backward select
def delay_seek_backword(sublist,delay_times,init_index):
    global max_r2
    global min_error
    global final_sublist
    if delay_times <= 0 or delay_times >= len(sublist):
        print("Error, delay_times must be a positive number!")
        return False,None,None
    print("enter seek")
    for i in reversed(range(init_index)):
        feature = sublist[i]
        r2,mape_err = remove_err(sublist,i)
        if mape_err < min_error and r2 > max_r2 :
            min_error = mape_err
            max_r2 = r2
            final_sublist = sublist[:]
            print(str(len(sublist)) + ",min_error=" + str(min_error) + ",max_r2=" + str(max_r2))
            return True, i, sublist
        else:
            sublist.insert(i,feature)
    delay_times -= 1
    if(delay_times > 0):
        feature = sublist[init_index]
        sublist.remove(feature)
        ok,index_return,list_return = delay_seek_backword(sublist,delay_times,init_index - 1)
        if ok:
            return ok,index_return,list_return
        else:
            sublist.insert(init_index,feature)
    return False,None,None

In [ ]:
#calculate the model error corresponding to feature subsets of different sizes, as a comparison, you can find a relatively stable and small error interval for feature selection
init_list = []
cur_error = 100
for i in range(len(features_list)):
    sublist = features_list[: i + 1]
    X = np.array(conduct[sublist])
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 32, shuffle = True)
    svr_pre = regr.fit(X_train,y_train).predict(X_test)
    mape_err = mean_absolute_percentage_error(y_test,svr_pre)
    if mape_err < cur_error:
        init_list.append(features_list[i])
    cur_error =  mape_err
    r2 = r2_score(y_test,svr_pre)
    print(str(mape_err) + ",len=" + str(len(sublist)) + "，r2=" + str(r2))

In [ ]:
X = np.array(conduct[init_list])
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 32, shuffle = True)
svr_pre = regr.fit(X_train,y_train).predict(X_test)
mape_err = mean_absolute_percentage_error(y_test,svr_pre)
mape_err

In [ ]:
for i in range(len(init_list)):
    features_list.remove(init_list[i])
print(len(features_list))
print(len(init_list))

In [ ]:
sublist = init_list
left_index = 70
delay_times = 6
min_error = 0.06
sublist = features_forward_select(sublist,delay_times)
print()
print(final_sublist)
print(len(final_sublist))
print(min_error)

In [ ]:
delay_times = 15
max_r2 = 0 
sublist = features_backward_select(sublist,delay_times)
print(final_sublist)
print(len(final_sublist))